### DS2002 Project 1 - Helena Moore 

#### Project overview
This project designs a dimensional data mart that represents the **business process** of choosing what films to stock based off of customer rental preferences. 

**Dimension tables:** dim_film *(film information)*, dim_customer *(customer information)*, dim_date *(date dimension which allows the analysis of business processes over time, namely in the fact_rental table)*

**Fact table:** fact_rental *(fact table which combines the dimension tables, shows the rental business process)*

#### Import the Necessary Libraries

In [2]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

#### Declare & Assign Connection Variables for the MySQL Server & Databases 

In [3]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "sakila"
dst_dbname = "sakila_dw"

#### Define Functions for Getting Data From and Setting Data Into Databases

In [4]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.

In [5]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(f"CREATE DATABASE IF NOT EXISTS`{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

### Create and populate the dimension tables

Create the film dimension table -- Join film with film_category, then join with category so that the category of the film is included in the 'film' dataframe.

In [6]:
sql_film = "SELECT * FROM sakila.film;"
df_film = get_dataframe(user_id, pwd, host_name, src_dbname, sql_film)
df_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 00:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 00:03:42


In [7]:
sql_film_category = "SELECT * FROM sakila.film_category;"
df_film_category = get_dataframe(user_id, pwd, host_name, src_dbname, sql_film_category)
df_film_category.head(2)

,film_id,category_id,last_update
0,1,6,2006-02-15 00:07:09
1,2,11,2006-02-15 00:07:09


In [8]:
sql_category = "SELECT * FROM sakila.category;"
df_category = get_dataframe(user_id, pwd, host_name, src_dbname, sql_category)
df_category.head(2)

,category_id,name,last_update
0,1,Action,2006-02-14 23:46:27
1,2,Animation,2006-02-14 23:46:27


In [9]:
df_dim_film = df_film
df_dim_film  = pd.merge(df_film, df_film_category, on='film_id', how='inner')
df_dim_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_x,category_id,last_update_y
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 00:03:42,6,2006-02-15 00:07:09
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 00:03:42,11,2006-02-15 00:07:09


In [10]:
df_dim_film  = pd.merge(df_dim_film, df_category, on='category_id', how='inner')
df_dim_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_x,category_id,last_update_y,name,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 00:03:42,6,2006-02-15 00:07:09,Documentary,2006-02-14 23:46:27
1,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 00:03:42,6,2006-02-15 00:07:09,Documentary,2006-02-14 23:46:27


#### Perform necessary tranformations to film dimension table
Rename id column to key, drop unneccessary columns

In [11]:
df_dim_film.rename(columns={"name":"category","film_id":"film_key"}, inplace=True)
df_dim_film.drop(['language_id','original_language_id','length','category_id','last_update_x','last_update_y','last_update'], axis=1, inplace=True)
df_dim_film.head(2)

,film_key,title,description,release_year,rental_duration,rental_rate,replacement_cost,rating,special_features,category
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,20.99,PG,"Deleted Scenes,Behind the Scenes",Documentary
1,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,7,2.99,18.99,NC-17,"Trailers,Deleted Scenes",Documentary


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases 

In [12]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "sakila_dw"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "cluster_name",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local",
    "db_name" : "sakila_mongo" #PUT THE NAME HERE 
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [13]:

def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data

In [14]:
client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"customer" : 'sakilacustomer.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

#### Extract data from source mongoDB into dataframe 
Getting the customer dataframe 

In [15]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customer"

df_customer = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customer.head(2)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-14 23:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-14 23:57:20


#### Create the customer dimension table

In [16]:
df_dim_customer = df_customer
df_dim_customer.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_dim_customer.drop(['email','address_id','last_update'], axis=1, inplace=True)
df_dim_customer.head(2)

,customer_key,store_id,first_name,last_name,active,create_date
0,1,1,MARY,SMITH,1,2006-02-14 22:04:36
1,2,1,PATRICIA,JOHNSON,1,2006-02-14 22:04:36


#### Create the rental fact table 

In [17]:
sql_rental = "SELECT * FROM sakila.rental;"
df_rental = get_dataframe(user_id, pwd, host_name, src_dbname, sql_rental)
df_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 16:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 16:30:53


In [18]:
sql_inventory = "SELECT * FROM sakila.inventory;"
df_inventory = get_dataframe(user_id, pwd, host_name, src_dbname, sql_inventory)
df_inventory.head(2)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 00:09:17
1,2,1,1,2006-02-15 00:09:17


In [19]:
df_fact_rental = df_rental 
df_fact_rental = pd.merge(df_fact_rental, df_inventory, on='inventory_id', how='inner')
df_fact_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update_x,film_id,store_id,last_update_y
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 16:30:53,80,1,2006-02-15 00:09:17
1,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,2006-02-15 16:30:53,80,1,2006-02-15 00:09:17


In [20]:
df_fact_rental = pd.merge(df_fact_rental, df_film, on='film_id', how='inner')
df_fact_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update_x,film_id,store_id,last_update_y,...,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 16:30:53,80,1,2006-02-15 00:09:17,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 00:03:42
1,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,2,2006-02-15 16:30:53,80,1,2006-02-15 00:09:17,...,2006,1,None,7,2.99,148,21.99,G,Trailers,2006-02-15 00:03:42


In [21]:
df_fact_rental.rename(columns={"rental_id":"rental_key","film_id":"film_key","customer_id":"customer_key"}, inplace=True)
df_fact_rental.drop(['last_update_x','last_update_y','staff_id','description','release_year','language_id','original_language_id','rental_duration','rental_rate','length','replacement_cost','rating','special_features','last_update'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_key,rental_date,inventory_id,customer_key,return_date,film_key,store_id,title
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,80,1,BLANKET BEVERLY
1,1577,2005-06-16 04:03:28,367,327,2005-06-24 22:40:28,80,1,BLANKET BEVERLY


#### Directions: run SQL script for creating and populating the date dimension table (midterm_Create_Populate_Dim_Date.sql) before next step

#### Get data from the date dimension table

In [22]:
sql_dim_date = "SELECT date_key, full_date FROM sakila_dw.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


#### Integrate the date dimension table

In [23]:
df_fact_rental_date = df_dim_date.rename(columns={"date_key" : "rental_date_key", "full_date" : "rental_date"})
df_fact_rental.rental_date = df_fact_rental.rental_date.astype('datetime64[ns]').dt.date
df_fact_rental = pd.merge(df_fact_rental, df_fact_rental_date, on='rental_date', how='left')
df_fact_rental.drop(['rental_date'], axis=1, inplace=True)
df_fact_rental.head(2)

,rental_key,inventory_id,customer_key,return_date,film_key,store_id,title,rental_date_key
0,1,367,130,2005-05-26 22:04:30,80,1,BLANKET BEVERLY,20050524
1,1577,367,327,2005-06-24 22:40:28,80,1,BLANKET BEVERLY,20050616


#### Load the dataframes into the data warehouse 

In [ ]:
db_operation = "insert"

tables = [('dim_film', df_dim_film, 'film_key'),
          ('dim_customer', df_dim_customer, 'customer_key')]


In [ ]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [24]:
db_operation = "insert"

tables = [('fact_rental', df_fact_rental, 'rental_key')]

In [25]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### SQL Queries (Select Statements) 

Query for the most popular films to rent -- fact_rental table

In [44]:
sql_rental_query = """
    SELECT title, COUNT(*) as amount_of_rentals FROM sakila_dw.fact_rental
    GROUP BY title
    ORDER BY amount_of_rentals DESC;
"""
df_rental_query = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_rental_query)
df_rental_query.head(5)

,title,amount_of_rentals
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,GRIT CLOCKWORK,32
3,RIDGEMONT SUBMARINE,32
4,JUGGLER HARDLY,32


Query for average rental rate grouped by category (most expensive category of film to rent) -- dim_film table

In [43]:
sql_film_query = """
    SELECT category, AVG(rental_rate) as avg_rate FROM sakila_dw.dim_film
    GROUP BY category
    ORDER BY avg_rate DESC;
"""
df_film_query = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_film_query)
df_film_query.head(10)

,category,avg_rate
0,Games,3.252295
1,Travel,3.235614
2,Sci-Fi,3.219508
3,Comedy,3.162414
4,Sports,3.125135
5,New,3.116984
6,Foreign,3.099589
7,Horror,3.025714
8,Drama,3.022258
9,Music,2.950784


Query for customers that rent out movies most frequently -- dim_customer table

In [42]:
sql_customer_query = """
  SELECT fact_rental.customer_key, COUNT(*) as times_rented, MAX(dim_customer.last_name) AS last_name
    FROM sakila_dw.fact_rental
    LEFT JOIN sakila_dw.dim_customer ON fact_rental.customer_key = dim_customer.customer_key
    GROUP BY fact_rental.customer_key
    ORDER BY times_rented DESC
    ;
"""
df_customer_query = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_customer_query)
df_customer_query.head(10)

,customer_key,times_rented,last_name
0,148,46,HUNT
1,526,45,SEAL
2,236,42,DEAN
3,144,42,SHAW
4,75,41,SANDERS
5,197,40,PETERS
6,469,40,BULL
7,468,39,CARY
8,137,39,KENNEDY
9,178,39,SNYDER
